## Boss Order

Task:

Our company is not working efficiently! We spend way too much drafting documents and this needs to be fixed!

For the company, you need to create an AI Agentic System that can speed up drafting documents, emails etc. The AI Agent System that can speed up drafting documents emails etc. The AI Agentic system should have Human-AI collabration meaning the human should be able to provide continuous feedback and AI Agent should stop if the human is happy with the Draft. The system should be able to fast and save the drafts

In [1]:
from typing import Annotated, Sequence, TypedDict
import os
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

load_dotenv()  # Load environment variables from .env file

# Initialize global document content
document_content = ""

# Check if API key is available
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("🔑 OpenAI API Key not found!")
    print("Please create a .env file in your project directory with:")
    print("OPENAI_API_KEY=your_actual_api_key_here")
    
    # For testing purposes, set a placeholder
    api_key = "placeholder"

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

@tool
def update(content: str) -> str:
    """Updates the current result with the provided content."""
    global document_content
    document_content = content
    return f"Document updated to: {document_content}"

@tool
def save(filename: str) -> str:
    """Saves the current document content to a file.

    Args:
        filename (str): The name of the file to save the document content to.

    Returns:
        str: A message indicating the result of the save operation.
    """
    global document_content

    if not filename.endswith(".txt"):
        filename = f"{filename}.txt"
    try: 
        with open(filename, "w") as f:
            f.write(document_content)
        print(f"Document saved to {filename}")
        return f"Document saved to {filename}"
    except Exception as e:
        print(f"Error saving document to {filename}: {e}")
        return f"Error saving document to {filename}: {e}"

tools = [save, update]

# Fix: Use 'model' instead of 'model_name' and use a more available model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=api_key).bind_tools(tools)

def our_agent(state: AgentState) -> AgentState:
    global document_content
    
    system_prompt = SystemMessage(
        content=f"""
        You are a drafter, a helpful writing assistant. You are going to help the user update and modify documents. 
        - If the user wants to update or modify content, use the 'update' tool with the complete updated content. 
        - If the user wants to save and finish, you need to use the 'save' tool with the filename.
        - Make sure to always show the current document state after modifications.
        Current document content: '{document_content}'
        """)
    
    if not state["messages"]:
        user_input = input("I am ready to help you update a document. What would you like to create? ")
        user_message = HumanMessage(content=user_input)
    else:
        user_input = input("\nWhat would you like to do with the document? ")
        user_message = HumanMessage(content=user_input)
    
    all_messages = [system_prompt] + list(state["messages"]) + [user_message]
    response = model.invoke(all_messages)

    print(f"\n=== Agent Response ===\n{response.content}\n")
    if hasattr(response, "tool_calls") and response.tool_calls:
        for tool_call in response.tool_calls:
            print(f"Invoking tool: {tool_call['name']} with args: {tool_call['args']}")
    
    return {"messages": [user_message, response]}

def should_continue(state: AgentState) -> str:
    """Determines whether the agent should continue or stop based on the latest message."""
    messages = state["messages"]
    if not messages:
        return "continue"
    
    # Check for tool messages indicating save completion
    for message in reversed(messages):
        if (isinstance(message, ToolMessage) and 
            "saved" in message.content.lower() and 
            "document" in message.content.lower()):
            return "end"
    
    return "continue"

def print_messages(messages):
    """Prints the sequence of messages in a readable format."""
    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"[ToolMessage] {message.content}")

# Fix: Proper graph construction
graph = StateGraph(AgentState)
graph.add_node("agent", our_agent)
graph.add_node("tools", ToolNode(tools))

graph.set_entry_point("agent")

# Fix: Add proper conditional edge from agent to tools
graph.add_conditional_edges(
    "agent",
    lambda state: "tools" if (state["messages"] and 
                             hasattr(state["messages"][-1], "tool_calls") and 
                             state["messages"][-1].tool_calls) else "continue",
    {
        "tools": "tools",
        "continue": "agent"
    }
)

graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        "continue": "agent",
        "end": END
    }
)

# Fix: Correct spelling
app = graph.compile()

def run_document_drafter():
    print("Welcome to the Document Drafter Agent!")
    print("Instructions:")
    print("- Ask me to create, update, or modify document content")
    print("- Ask me to save the document when you're satisfied")
    print("- The agent will help you iteratively improve your document\n")

    state = {"messages": []}

    try:
        for step in app.stream(state, stream_mode="values"):
            if "messages" in step:
                print_messages(step["messages"])
    except Exception as e:
        print(f"Error occurred: {e}")
    
    print("Document Drafter Agent has finished.")

# Only run if we have a proper API key
if api_key and api_key != "placeholder":
    run_document_drafter()
else:
    print("\n⚠️  Cannot run without OpenAI API key!")
    print("Create a .env file with your API key to use this document drafter.")

Welcome to the Document Drafter Agent!
Instructions:
- Ask me to create, update, or modify document content
- Ask me to save the document when you're satisfied
- The agent will help you iteratively improve your document


=== Agent Response ===


Invoking tool: update with args: {'content': 'I am planning to write about a movie which I just saw today featuring Brad Pitt in the role of Death.'}
[ToolMessage] Document updated to: I am planning to write about a movie which I just saw today featuring Brad Pitt in the role of Death.

=== Agent Response ===
Please provide the name of the movie so I can update the document accordingly.

[ToolMessage] Document updated to: I am planning to write about a movie which I just saw today featuring Brad Pitt in the role of Death.

=== Agent Response ===
I need the name of the movie to update the document. Please provide the movie name.


=== Agent Response ===


Invoking tool: update with args: {'content': "I am planning to write about a movie which I